In [ ]:
def rebalance_position1(self):
    stock = 'IM.CFE'
    stock_list_portfolio = list(self.positions_dict['long'].keys())
    close_series = (self.stock_dict_data.xs('IM.CFE', level='code')).loc[:self.current_date,'close']
    prices = close_series
    std_1 = prices.rolling(6).std()
    max_5_percentile = std_1.quantile(0.90)
    for stock in stock_list_portfolio:
        self.order_target(stock, self.positions_dict['long'][stock]['amount'])
    total_value = self.cash_value()
    if max_5_percentile < std_1.iloc[-2] and std_1.iloc[-1] < std_1.iloc[-2] and std_1.iloc[-3] < std_1.iloc[-2]:
        order = self.order_value(stock, total_value)

In [ ]:
def first_exit_time_volatility(prices, delta,q=5):
    log_prices = prices
    exit_times = []
    i = 0
    n = len(log_prices)
    while i < n - 1:
        base = log_prices[i]
        j = i + 1
        while j < n:
            if abs(log_prices[j] - base) >= delta:
                exit_times.append(j - i)
                i = j  # 重设区间
                break
            j += 1
        else:
            break  # 没有再越过 delta，结束
    if not exit_times:
        raise ValueError("没有触发任何一次退出事件，请增大样本或减小 ∆")
    tau_q1 = np.percentile(exit_times, q)
    k = 1
    while k < len(log_prices):
        prices_delta = log_prices[-1] - log_prices[-1 - k]
        if abs(prices_delta) >= delta or k > tau_q1:
            break
        k += 1
    return k <= tau_q1,prices_delta

#同步增加
def generate_linear_position_rules(start=5000, end=6000, steps=5):
    price_step = (end - start) / steps
    rules = []
    for i in range(steps + 1):
        price = round(start + i * price_step, 3)
        rules.append(price)
    return rules
#同步减小
def generate_reverse_linear_position_rules(start=5000, end=6000, steps=5, max_fund=20000):
    price_step = (end - start) / steps
    fund_step = max_fund / steps
    rules = []
    for i in range(steps + 1):
        price = round(start + i * price_step, 3)
        fund = max_fund - i * fund_step
        rules.append((price, fund))
    return rules

from datetime import timedelta

def calc_avg_option_delta(self, stock, option_code, now, minutes=10):
    """
    计算过去 N 分钟内的 options_delta 平均值，并根据期权类型（Call/Put）过滤异常 delta 值。

    Parameters:
    - stock: 标的代码
    - option_code: 期权代码（需包含 `-C-` 或 `-P-` 标识）
    - now: 当前时间（datetime 类型）
    - minutes: 回溯分钟数，默认 10 分钟

    Returns:
    - 平均 delta（float），Call 返回正数，Put 返回负数
    - 如果无法计算，则返回 None
    """
    # 判断期权类型（检查 `-C-` 或 `-P-`）
    if "-C-" in option_code.upper():
        option_type = "CALL"
    elif "-P-" in option_code.upper():
        option_type = "PUT"
    else:
        raise ValueError(f"期权代码 {option_code} 不包含 `-C-` 或 `-P-`，无法识别 Call/Put")

    delta_list = []
    for i in range(minutes, 0, -1):
        t_now = now - timedelta(minutes=i - 1)
        t_past = now - timedelta(minutes=i)
        try:
            option_now = self.get_price(t_now, option_code)
            option_past = self.get_price(t_past, option_code)
            stock_now = self.get_price(t_now, stock)
            stock_past = self.get_price(t_past, stock)

            # 跳过无效数据
            if None in (option_now, option_past, stock_now, stock_past):
                continue
            if stock_now == stock_past:
                continue

            delta = (option_now - option_past) / (stock_now - stock_past)

            # 根据期权类型过滤 delta
            if option_type == "CALL" and delta >= 0:  # Call 期权 delta 应为正
                delta_list.append(delta)
            elif option_type == "PUT" and delta <= 0:  # Put 期权 delta 应为负
                delta_list.append(delta)  # 取绝对值方便计算
        except Exception as e:
            self.log_back.append(f"[Delta计算异常] {t_past}~{t_now}: {e}")
            continue
    
    if delta_list:
        avg_delta = sum(delta_list) / len(delta_list)
        if abs(avg_delta) > 1 :
            print(now,'-',delta_list)
        return avg_delta  # Put 返回负数
    else:
        raise ValueError("计算错误：未获得有效 delta 数据")

#计算delta
def calculate_delta(S, K, T, r, sigma, option_type='call'):
    """
    计算欧式股指期权价格（Black模型，适用于现金交割）
    :param S: 标的指数当前价格
    :param K: 期权执行价
    :param T: 剩余期限（年）
    :param r: 无风险利率（年化）
    :param sigma: 隐含波动率（年化）
    :param option_type: 'call'（看涨）或 'put'（看跌）
    :return: 期权价格
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        delta = np.exp(-r * T) * norm.cdf(d1)
    else:
        delta = np.exp(-r * T) * (norm.cdf(d1) - 1)
    return delta

#不依赖外部库的Gamma计算
def calculate_gamma_independent(S, K, T, r, sigma, is_future=True, q=0.0):
    """
    计算欧式股指期权价格（Black模型，适用于现金交割）
    :param S: 标的指数当前价格
    :param K: 期权执行价
    :param T: 剩余期限（年）
    :param r: 无风险利率（年化）
    :param sigma: 隐含波动率（年化）
    :param option_type: 'call'（看涨）或 'put'（看跌）
    :return: 期权价格
    """
    # 边界条件处理
    if S <= 0 or T <= 0 or sigma <= 0:
        return 0.0
    # 添加极小值避免数值问题
    T = max(T, 1e-6)
    sigma = max(sigma, 1e-4)
    # 处理期货期权特例
    if is_future:
        q = r
    # 计算d1
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    # 计算标准正态分布的概率密度函数
    phi_d1 = (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * d1**2)
    # 计算Gamma
    gamma = np.exp(-q * T) * phi_d1 / (S * sigma * np.sqrt(T))
    return gamma

In [ ]:
#V5做多完整版
#1、当出现入场信号买期权，和一成标的，加大高抛低吸阈值
def rebalance_position(self):
    global option_code
    global middle_close #记录止盈的价格
    global K
    global r
    global sigma
    global T
    stock = 'IM.CFE'
    df_stock = self.stock_dict_data.xs(stock, level='code')
    close_series = df_stock.loc[:self.current_date, 'close']

    if len(close_series) < 10:
        return
    #首次买入记录一半持仓long_amount
    if self.count == 0:
        if self.current_date in date_list_5m:
            # 找到波幅开始的价格
            global df_5m
            df_middle = df_5m.xs(stock, level='code')
            df_middle = df_middle.loc[:self.current_date, 'close'].copy()
            df_std = df_middle.rolling(6).std()
            std_mean= df_std.rolling(120).mean()
            condition = df_std < std_mean
            start_date = df_middle[condition].index[-1]
            start_close = self.get_price(start_date, stock)
            new_close = self.get_price(self.current_date, stock)
            middle_close = (start_close+new_close)/2 #计算止盈价格
            
            #判断方向
            prices = df_middle[-11:]  # 取最近10个交易日的收盘价
            std = prices.rolling(9).std()
            delta = std.iloc[-1] * 2
            prices = df_middle.values
            result1, result2 = first_exit_time_volatility(prices, delta, q=50)
            if result2 > 0:
                self.count = 1 #记录上涨做空开仓信号
                print(f"{self.current_date}做空开仓信号，平仓价格：",middle_close)
            else:
                self.count = 2 #记录下跌做多开仓信号
                print(f"{self.current_date}做多开仓信号，平仓价格：",middle_close)
        else:
            return

    new_price = self.get_price(self.current_date, stock)
    if (self.current_date.time() == pd.to_datetime('14:57:00').time() or (
        (self.count == 1 and new_price < middle_close) or (self.count == 2 and new_price > middle_close)
        )) and (self.positions_dict['short'] or self.positions_dict['long']): #平仓条件
        print(self.current_date)
        if self.count == 1:  # 上涨平仓
            self.close_short(stock, self.positions_dict['short'][stock]['amount'])
            code1 = list(self.positions_dict['long'].keys())[0]  # 获取期权代码
            self.order_target(code1, self.positions_dict['long'][code1]['amount'])
            #print(self.positions_dict)
        elif self.count == 2:  # 下跌平仓
            self.order_target(stock, self.positions_dict['long'][stock]['amount'])
            code1 = list(self.positions_dict['long'].keys())[0]  # 获取期权代码
            self.order_target(code1, self.positions_dict['long'][code1]['amount'])
            #print(self.positions_dict)
        # 重置状态 
        self.k = 0 
        self.count = 0 
        self.long_dict = {"long_value":0,'cost_price':0,'cash_value':0,'amount':0}
        return
    
    if self.count == 1:  # 做空开仓
        if stock not in self.positions_dict['short']:
            prices = close_series
            if self.k == 0 and prices.iloc[-1] < prices.iloc[-2]:
                self.k = 1
                return
             
            # 获取期货合约的收盘价,查找接近虚值期权合约
            strike = self.get_price(self.current_date, '000852.SH')
            strike_rounded = round(strike, 2)
            strike_rounded = int(math.ceil(strike / 100.0)+1) * 100  #向上取整
            ym_str = self.current_date.strftime('%y%m')
            option_code = f"MO{ym_str}-C-{strike_rounded}.CFE"

            #获取到期日
            a = w.wsd(option_code, "startdate,lasttradingdate,us_impliedvol", self.current_date, self.current_date, "")
            start_date = a.Data[0][0]
            expiration_date = a.Data[1][0]#到期日
            option_iv = a.Data[2][0]
            
            if option_iv == None or option_iv > 0.3: #当期权IV较大时不做
                self.count = 0
                return
            if expiration_date == None: #如果没有这个期货则跳过
                self.count = 0
                return
            remaining_days = (expiration_date - self.current_date).days
            start_days = (self.current_date - start_date).days #起始交易日在该日期之后
            if remaining_days < 0 or remaining_days > 20 or start_days < 10: #当没有期权满足这个条件时就不做交易
                self.count = 0
                return
            
            S = self.get_price(self.current_date, '000852.SH') #指数价格
            K = strike_rounded #行权价
            r = 0.015
            sigma = option_iv
            T = (remaining_days+1)/252
            gamma = calculate_gamma_independent(S, K, T, r, sigma)
            
            cash_5m = self.cash_value()
            self.open_short(stock, cash_5m*0.45) #买入期货多头
            
            options_amount = self.positions_dict['short'][stock]['amount'] * - 2 #计算期权买入量
            #option_code = self.find__option(self.current_date, 'put') #查找平值期权
            
            options_value = self.get_price(self.current_date, option_code) * options_amount
            self.order_value(option_code, options_value) #买入看跌期权
            print(f"当前日期：{self.current_date}，期权代码：{option_code}，期货数量：{self.positions_dict['short'][stock]['amount']}，期权数量：{self.positions_dict['long'][option_code]['amount']}")
            self.long_dict["long_value"] = int((self.positions_dict['short'][stock]['amount'])) #记录底仓持仓量
            stock_prices = self.get_price(self.current_date, stock)
            self.long_dict['cash_value'] = generate_linear_position_rules(start=middle_close,
                                                                          end=(2*stock_prices-1*middle_close),
                                                                          steps=10, max_fund=cash_5m*0.45)   #生成加仓列表
            _,value = min(self.long_dict['cash_value'], key=lambda x: abs(x[0] - stock_prices))
            options_delta = calculate_delta(S, K, T, r, sigma, option_type='call')
            stock_delta = options_delta*self.positions_dict['long'][option_code]['amount'] + 2*self.positions_dict['long'][option_code]['amount']*gamma 
            stock_value = stock_delta * self.get_price(self.current_date , stock)
            self.close_short(stock,self.positions_dict['short'][stock]['amount'])
            print(stock_value,'-',options_delta)
            self.open_short(stock, stock_value)
            self.long_dict['cost_price'] = value #记录状态
            self.log_back.append(f"")
            return 
        else:
            stock_prices = self.get_price(self.current_date, stock)
            price,value = min(self.long_dict['cash_value'], key=lambda x: abs(x[0] - stock_prices))
            #options_delta = (self.get_price(self.current_date,option_code) - self.get_price(self.past_date,option_code))/(self.get_price(self.current_date,stock)-self.get_price(self.past_date,stock))
            S = self.get_price(self.current_date, '000852.SH') #指数价格
            gamma = calculate_gamma_independent(S, K, T, r, sigma)  
            options_delta = calculate_delta(S, K, T, r, sigma, option_type='call')
            self.log_back.append(f"当前持仓：{self.positions_dict['short'][stock]['amount']} delta：{options_delta} ，S：{S} T：{T} R：{r} sigma:{sigma}")
            if self.long_dict['cost_price'] != value:
                stock_delta = options_delta*self.positions_dict['long'][option_code]['amount'] + 2*self.positions_dict['long'][option_code]['amount']*gamma 
                if (stock_delta >= self.positions_dict['short'][stock]['amount'] and stock_prices > price) or (stock_delta <= self.positions_dict['short'][stock]['amount'] and stock_prices < price):
                    stock_value = stock_delta * self.get_price(self.current_date , stock)
                    self.close_short(stock,self.positions_dict['short'][stock]['amount'])
                    self.open_short(stock, stock_value)
                    self.long_dict['cost_price'] = value #更新状态

    elif self.count == 2:  # 做多开仓
        if stock not in self.positions_dict['long']:
            prices = close_series
            if self.k == 0 and prices.iloc[-1] > prices.iloc[-2]:
                self.k = 1
                return
            
            # 获取期货合约的收盘价,查找接近虚值期权合约
            strike = self.get_price(self.current_date, '000852.SH')
            strike_rounded = int(math.floor(strike / 100.0)-1) * 100  # 向下取整到最近的200
            ym_str = self.current_date.strftime('%y%m')
            option_code = f"MO{ym_str}-P-{strike_rounded}.CFE"
            
            #获取到期日
            a = w.wsd(option_code, "startdate,lasttradingdate,us_impliedvol,gamma", self.current_date, self.current_date, "")
            start_date = a.Data[0][0]
            expiration_date = a.Data[1][0] #到期日
            option_iv = a.Data[2][0]
            gamma = a.Data[3][0]
            if option_iv == None or option_iv > 0.3: #当期权IV较大时不做
                self.count = 0
                return
            if expiration_date == None: #如果没有这个期货则跳过
                self.count = 0
                return
            remaining_days = (expiration_date - self.current_date).days
            start_days = (self.current_date - start_date).days
            if remaining_days < 0 or remaining_days > 20 or start_days < 10: #当没有期权满足这个条件时就不做交易
                self.count = 0
                return
                
            S = self.get_price(self.current_date, '000852.SH') #指数价格
            K = strike_rounded #行权价
            r = 0.015
            sigma = option_iv
            T = (remaining_days+1)/252
            gamma = calculate_gamma_independent(S, K, T, r, sigma)

            cash_5m = self.cash_value()
            self.order_value(stock, cash_5m*0.45) #买入期货多头
            options_amount = self.positions_dict['long'][stock]['amount'] * 2 #计算期权买入量
            #option_code = self.find__option(self.current_date, 'put') #查找平值期权

            options_value = self.get_price(self.current_date, option_code) * options_amount
            self.order_value(option_code, options_value) #买入看跌期权
            #print(self.positions_dict)
            print(f"当前日期：{self.current_date}，期权代码：{option_code}，期货数量：{self.positions_dict['long'][stock]['amount']}，期权数量：{self.positions_dict['long'][option_code]['amount']}")
            self.long_dict["long_value"] = int((self.positions_dict['long'][stock]['amount'])) # 记录底仓持仓量
            stock_prices = self.get_price(self.current_date, stock)
            self.long_dict['cash_value'] = generate_reverse_linear_position_rules(start=(2*stock_prices-1*middle_close), 
                                                                                end=middle_close,
                                                                                steps=10, max_fund=cash_5m*0.45)   #生成加仓列表
            _,value = min(self.long_dict['cash_value'], key=lambda x: abs(x[0] - stock_prices))
            #options_delta = (self.get_price(self.current_date,option_code) - self.get_price(self.past_date,option_code))/(self.get_price(self.current_date,stock)-self.get_price(self.past_date,stock))
            options_delta = calculate_delta(S, K, T, r, sigma, option_type='put')
            stock_delta = -options_delta*self.positions_dict['long'][option_code]['amount'] + 2*self.positions_dict['long'][option_code]['amount']*gamma 
            stock_value = stock_delta * self.get_price(self.current_date , stock)
            print(stock_value,'-',options_delta)
            self.order_target(stock,self.positions_dict['long'][stock]['amount'])
            if stock_value < 0 :
                stock_value
            self.order_value(stock, stock_value)
            self.long_dict['cost_price'] = value #记录状态
            self.log_back.append(f"")
            return 
        
        else:
            stock_prices = self.get_price(self.current_date, stock)
            price,value = min(self.long_dict['cash_value'], key=lambda x: abs(x[0] - stock_prices))
            S = self.get_price(self.current_date, '000852.SH') #指数价格
            gamma = calculate_gamma_independent(S, K, T, r, sigma)  
            options_delta = calculate_delta(S, K, T, r, sigma, option_type='put')
            self.log_back.append(f"当前持仓：{self.positions_dict['long'][stock]['amount']}，delta：{options_delta} ，S：{S} T：{T} R：{r} sigma:{sigma}")
            if self.long_dict['cost_price'] != value:
                stock_delta = -options_delta*self.positions_dict['long'][option_code]['amount'] + 2*self.positions_dict['long'][option_code]['amount']*gamma 
                if (stock_delta >= self.positions_dict['long'][stock]['amount'] and stock_prices < price) or (stock_delta <= self.positions_dict['long'][stock]['amount'] and stock_prices > price):
                    stock_value = stock_delta * self.get_price(self.current_date , stock)
                    self.order_target(stock,self.positions_dict['long'][stock]['amount'])
                    self.order_value(stock, stock_value)
                    if stock_value < 0:
                        raise ValueError("买入资金为负值")
                    self.long_dict['cost_price'] = value #更新状态

In [11]:
option_code = 'MO2507-P-6300'
option_type = "CALL" if "-C-" in option_code else ("PUT" if "-P-" in option_code else None)

In [12]:
option_type

'PUT'